<a href="https://colab.research.google.com/github/gregorimaia/Engenharia_Aeroespacial/blob/main/Analise_sistemas_realimentados_e_estabilidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install control
import numpy as np
from control.matlab import *
import matplotlib.pyplot as plt
import control

# Análise de sistemas realimentados

Sistemas de Controle I - Engenharia Aeroespacial - UFSM

Prof. Dr. Lucas Vizzotto Bellinaso



Insira abaixo a FTMA que deseja testar:


In [ ]:
#Insira a Função de Transferência de Malha Aberta  (C*Gp)
FTMA = tf(20.3,[1, 27.6, 98.28])    #caso 1
#FTMA = tf(20,[1, 4, 3, 1])  #caso 2
print(f'Função de Transferência de Malha Aberta: \n{FTMA}')


#Cálculo da Função de Transferência de Malha Fechada:
FTMF = feedback(FTMA)
print(f'\nFunção de Transferência de Malha Fechada: \n{FTMF}')
print(f'Polos em Malha fechada: \n {FTMF.pole()}')
if (any(np.real(FTMF.pole())>0)): print('O sistema é INSTÁVEL em malha fechada\n')
else: print('\nO sistema é ESTÁVEL em malha fechada\n')

## 1 Erros em regime permanente para resposta ao degrau e à rampa


In [ ]:
#Inserir valores de r0 (degrau) e r1 (rampa):
r0 = 1
r1 = 1

# Cálculos da FTMF:
tau_FTMF = 1/np.min(-np.real(FTMF.pole()))
t_vec = np.linspace(0,5*tau_FTMF, 201)
stepr0 = np.full_like(t_vec, r0)
rampar1 = t_vec*np.full_like(t_vec, r1)
y_step,_,_ = lsim(FTMF, U=stepr0, T=t_vec)
y_ramp,_,_ = lsim(FTMF, U=rampar1, T=t_vec)

#Cálculo do erro:
kss = dcgain(FTMA);
kv = dcgain(minreal(FTMA*tf('s'), verbose=False));
np.seterr(divide='ignore')
ess = r0/(1+kss)
ev = r1/kv
print(f'Kss = {kss:.4f}\t\tKv = {kv:.4f}')
print(f'\nErro para resposta ao degrau r(t) = {r0:.2f} \tess = {ess:.4f}')
print(f'\nErro para resposta à rampa r(t) = {r1:.2f}t \tev = {ev:.4f}\n')

#Print das respostas:
plt.figure(figsize = (10,6))
plt.subplot('211')
plt.plot(t_vec, stepr0, t_vec, y_step)
plt.title(f'Resposta ao degrau em malha fechada. Regime permanente: ess = {ess:.4f}')
plt.legend(('Referência r', 'Saída y'))
plt.grid(); plt.ylabel('r e y');
plt.subplot('212')
plt.plot(t_vec, rampar1, t_vec, y_ramp)
plt.title(f'Resposta à rampa em malha fechada. Regime permanente: ev = {ev:.4f}')
plt.legend(('Referência r', 'Saída y'));
plt.grid(); plt.xlabel('tempo (s)'); plt.ylabel('r e y');


# 2 Estabilidade

## 2.1 Análise da Equação característica e polos em malha **fechada**

In [ ]:
polos_MF = FTMF.pole()
print(f'Polos em MF: {polos_MF}\n')
num,den = tfdata(FTMF);
EC = den[0][0]

print(f'Polinômio  característico: {EC}')
if any(np.real(polos_MF)>0): print('O sistema é INSTÁVEL')
else: print('O sistema é ESTÁVEL')

plt.figure(figsize=(10,6))
yout,t = step(FTMF)
plt.plot(t,yout);
plt.grid();  plt.title('Resposta ao degrau em malha fechada');

### 2.1.1 Análise dos polos em MF em função de K

In [ ]:
K = np.arange(1,20.1,1)
G = tf(1,[1, 4, 3, 1])

print(f'Valores de K = {K}')

print(f'\nG(s) = {G}')
print('FTMA = K*G(s)')

for k in K:
  FTMF0 = feedback(k*G)
  p1,p2,p3 = FTMF0.pole()
  if any(np.real([p1,p2,p3])>0): sest = 'INSTÁVEL'
  else: sest = 'Estável'
  print(f'K = {k:.0f}\t polos_MF = [{p1:.2f}, {p2:.2f}, {p3:.2f}]\t'+sest)
  plt.figure(figsize=(6,1))
  yout,t = step(FTMF0)
  plt.plot(t,yout);  plt.grid();
  plt.xlabel('time(s)'); plt.ylabel('y');
  plt.title(f'Step response: K = {k} - '+sest)

# 2.2 Critério de Nyquist

In [ ]:
K = np.array([1, 9, 10, 11, 12, 20])
G = tf(1,[1, 4, 3, 1])

print(f'Valores de K = {K}')

print(f'\nG(s) = {G}')
print('FTMA = K*G(s)')

for k in K:
  plt.figure(figsize=(10,5)); 
  plt.subplot('121');  plt.grid();
  plt.title(f'K = {k}')
  nyquist(k*G)
  plt.subplot('122')
  plt.title('Zoom em torno de (-1, 0)')
  nyquist(k*G)
  plt.ylim((-0.5, 0.5))
  plt.xlim((-1.5, -0.5))


## 2.3 Diagrama de Bode

In [ ]:
import matplotlib
matplotlib.use('nbagg')

G = tf(1,[1, 4, 3, 1])
k = 2
FTMA = k*G
print(f'\nG(s) = {G}')
print(f'K = {k}')
print(f'FTMA = K*G(s) = {k}*G(s)')

plt.figure(figsize=(8,8))
bode(FTMA);
GM,PM,wg,wc = margin(FTMA)
GMdB = mag2db(GM)
print(f'\n\nMargem de ganho em dB:  GM = {GMdB:.2f} dB')
print(f'Margem de fase em graus:  PM = {PM:.2f}°')
print('O sistema aparenta ser', 'ESTÁVEL.' if GMdB>0 and PM>0 else 'INSTÁVEL.')
print(f'Frequência de cruzamento por 0 dB:  wc = {wc:.2f} rad/s')

### 2.3.1 Margem de fase $\times$ coeficiente de amortecimento $\zeta$
Válido para sistems de 2ª ordem.

In [ ]:
PMdeg = np.arange(5,78,1)
PM = np.deg2rad(PMdeg)
Q = np.sqrt(np.cos(PM))/np.sin(PM)   
csi = 1/(2*Q)      # Q = 1/(2*csi)
plt.figure(figsize=(8,6))
plt.plot(PMdeg,csi)
plt.grid()
plt.title('Margem de fase em função do coeficiente de amortecimento')
plt.xlabel('Phase Margin (°)'); plt.ylabel('csi');
plt.ylim((0,1));  plt.xlim((5,80));

## 2.4 Lugar das raizes no SISOApp

In [ ]:
# Copy and run in the first cell:
!pip install bokeh             # Bokeh package must be installed in Google Colab server
!pip install control           # Control package must be installed in Google Colab server
!git clone https://github.com/lucasbellinaso/PythonSisoDesignApp.git
import os
os.chdir("PythonSisoDesignApp")
!python classes.py             # running the github code
from classes import SISOApp    # importing the SISOApp
from control.matlab import *   # importing control package as a Matlab environment
#help(SISOApp)

In [ ]:
FTMA = tf(1,[1, 4, 3, 1])    #caso 1
SISOApp(FTMA)